<img src="https://github.com/christopherhuntley/BUAN6510/blob/master/img/Dolan.png?raw=true" width="180px" align="right">

# **BUAN 6510**
# **Lesson 10: Dimensional Data Warehouse Design** 
_Data Driven Analytics_

## **Learning Objectives**
### **Theory / Be able to explain ...**
- How database design for analytical applications is different from that for transaction systems
- The various forms of the star schema and when each is most applicable
- The concept of data granularity in dimensional data warehouses
- Data cubes as a dimensional data model
- Recent SQL extensions geared towards analytical applications 

### **Skills / Know how to ...**
- Identify dimensions that provide context to facts
- Use SQL window functions to work with longitudinal data
- Use SQL arrays and structs to work with repeating groups and hierarchical data structures

--------
## **LESSON 10 HIGHLIGHTS**

In [ ]:
#@title Run this cell if video does not appear
%%html
<div style="max-width:1000px">
  <div style="position: relative;padding-bottom: 56.25%;height: 0;">
    <iframe style="position: absolute;top: 0;left: 0;width: 100%;height: 100%;" rel="0" modestbranding="1"  src="https://www.youtube.com/embed/ZSVRiOfodDY" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
  </div>
</div>

## **BIG PICTURE: The Holy Grail of Data Warehousing**

Data Warehousing as a universal data repository has been a dream since Ralph Kimball and Bill Inmon popularized the term in the 1980s. Analysts had by that point built up quite a repertoire of models for just about any kind of analysis. They could create classification and regression trees (decision trees, random forests, etc.). They could do linear, nonlinear, kernel, and logistic regression with large datasets. They could solve optimization problems with thousands of variables and tens of thousands of constraints. Even the neural network models at the core of the latest and greatest deep learning techniques were pretty mature by 1994.  

What was missing was data! Well, sort of. What we had was lots of transactional data locked up in siloed mainframe systems. Online banking (via ATMs) had been a reality since the 1970s, credit card transactions since the 1960s, and the air traffic control system since the 1950s. These systems were great at capturing event data, one event at a time. They could even handle bulk transactions and reporting, if you could wait for the job to run overnight. And heaven forbid that you might want data in a different format or that wasn't already included in a standard report.  

However, analytical data is not the same as transactional data. Among the differences: 
- **Temporal Scope:** Emphasis on historical data rather than real-time operations
- **Diversity:** Drawing on multiple sources instead of a single transaction system
- **Data Quality:** "good enough to run the company" was not good enough for analytical modeling
- **Performance Tradeoffs:** Focus on computational speed (on desktop computers) instead of data throughput (on shared mainframes)
- **Access:** Providing analysts with direct access to transaction systems is both insecure and expensive

Even back then the ultimate solutions were known, though not remotely close to being available. Consider, for example, this figure from Kimball's seminal book *The Data Warehouse Lifecycle Toolkit*, published in 1998 and based on original work **from the mid-1980s**.

![Kimball's Data Warehouse](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L10_Kimball_Data_Warehouse_Elements.png)

All of the elements of a modern data pipeline are there. It even articulated the steps of the ETL process in detail. It was all there ... to be realized *someday*. 

Someday is now. With commodity data storage, ample computing power, ready-made software for just about any kind of modeling, and the analytical results to attract attention from management, data infrastructure is finally seen as what it should have been all along: a critical resource upon which the company relies to make it stand out against the competition. 

That's what the vendors tell us, anyway. 

In this lesson we will explore the **Dimensional Data Warehouse** model first proposed by Kimball et al. all those years ago. We will also consider how its mass adoption has influenced the SQL standard in recent years, with the addition of extensions like  **arrays**, **structs**, and  **partitions** that relax fundamental assumptions of the relational data model. 

---
## **The Star Schema Pattern**

The star schema used by dimensional data warehouses is what we call a **design pattern**, a standard solution to a standard problem. By giving each pattern a name, designers can discuss their work with others without having to explain their decisions anew each time. As design patterns pass into common usage, they form a kind of shorthand design language that works in a given problem domain. 

The star schema pattern addresses the disconnect between the way data is recorded versus how it is used by analysts. It reduces all data down to measures (facts) and labels (dimensions). Since *all information* takes that form eventually anyway, the pattern strikes a nice balance between structure and general applicability. 

In our dimensional model, we end up with several moderately-sized **dimension tables** surrounding a large (possibly massive) **fact table**:
- The dimensions are somewhat timeless and immutable. They define the **context** for the facts. Even when the facts themselves may change over time, the dimensions remain relatively static. 
- The facts are somewhat volatile, with new **measures** continually added and others redefined to suit the ever changing needs of the analysts. If there is a way to precompute a statistic or other measure so analysts don't have to, then do it. If a given measure is no longer needed or misleading, then redefine or remove it. 

Once again, here is the NBA PlayFacts warehouse, this time noting some of the key features. We will use is as an example,starting with the dimensions before moving on to the facts. We will also explore variations on the general star schema pattern that fit certain use cases. 
![NBA PlayFacts Dim DW](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L10_Star_Schema_Notes.png)

### **Dimension Tables**
Dimensions represent the contextual language that we use to describe a given fact. In other words, the dimensions define the lens through which we view each fact. They are what give it context and meaning. 

In theory the **dimensions exist before the fact data is collected**. So, how do we know what they are before we have any data? A good starting point is the [Five Ws framework](https://en.wikipedia.org/wiki/Five_Ws) used by journalists and storytellers the world over:
- **Who was involved?** People, roles, etc. 
- **What happened** Event types, outcomes, etc. 
- **When did it happen?** Timing or place in a sequence
- **Where did it happen?** Location, which may be conceptual rather than physical
- **Why did it happen?** Intent, cause, etc. 
- **How did it happen?** Steps, sequential logic, etc. 

Though there is some disagreement about this, the usual recommendation is that dimension tables be fully denormalized. Since they are often fairly small (relative to the facts table) and don't change much, there is little chance of creating anomalies over time. So, while it may be tempting to, for example, normalize out zip codes and cities from a location dimension, there is no real need, especially when it would require an unnecessary table join.

As we can see in the NBA example, the first question $-$ who is involved in a given play $-$ is answered with three dimensions: 
- the individual player who gets credited with each event
- the lineup of players on the court at the time
- the team whose play is being reflected by the facts

It is done this way to support different, independently-calculated measures:
- the counting stats (points, rebounds, assists, etc.) for an individual player, lineup, or team
- the total playing time for each player or lineup  on the court, even when they are not generating counting stats

An often overlooked but potentially tricky aspect of dimension design is whether dimensions are allowed to overlap. In other words, can the same dimension be represented two different ways? Can we combine dimensions to create a third uber-dimension? 

Like a lot of things, it depends. For example, we could combine geo-location data (addresses) with organizational hierarchy (offices, districts, regions, etc.) into a single dimension if that is how the data is usually grouped. The result is several levels of **granularity**, all stored in one dimension table. For a different analytical application, however, we may want to keep things more separated, especially if the geographic locations overlap. It may be that, for example, the same physical location may mix personnel from multiple functions or divisions within a company. Then it would not make sense to treat locations as nodes in the organizational hierarchy. 

We will go into this idea more in the Pro Tips section, where we will discuss the peculiar logic of longitudinal segmentation (i.e., slicing time).



 

### **Fact Tables**
Fact tables exist at the intersection of the dimension tables. Each fact is labeled with foreign keys to the dimension tables, usually one key per dimension. The rest of the columns are measures that can be used in aggregate calculations.  

What makes a good measure? Anything from which we can calculate a descriptive statistic: 
- For text data, we generally are limited to the text itself and counts of some sort. We may, for example, count the number of times the word "no" appears, how many sentences there are, etc. 
- For numerical data we can use all of the usual statistics like mean, maximum, minimum, etc. 
- For temporal data (dates and times), we may calculate elapsed times, inter-event times, cumulative times, etc. that can be treated like numerical data
- For binary data (pictures, etc.), the options are very limited, though one may be able to apply a machine learning technique to generate numerical digests that can be aggregated.  

Interestingly, the measures can only be as granular as the dimensions allow. In other words, if a given dimension only has 3 possible labels, then that dimension can only divide up the facts three ways. Each additional dimension, however, increases the granularity accordingly. If every dimension were to have two possible values, then a one single dimension could divide the facts into 2 groups. With two dimensions, we could divide into up to 4 groups. With three dimensions, we could generate up to 8 groups, etc. 

One way to visualize this is with a (hyper-) cube, with each dimension on a side. Each fact is *binned* inside one of the smaller cubes at the intersection of the dimensions. For the NBA PlayFacts cube below, each fact is binned based on the game, team, and player. Thus, if with only three dimensions, we would only be able to generate box score stats for full games. In order to get statistics within a game (e.g., for the last two minutes of each period) we would need to include the play segment dimension. (Don't ask about how we'd show a 4-dimensional cube. Just know that we can.)

![Data Cube](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L10_DataCube_wide.png)

> **Heads Up:** It is sometimes difficult to distinguish dimensions from measures when source data is numerical. For example, is the time on the clock (i.e., seconds remaining in the period) a measure or a dimension? It is a measure, in the sense that it captures the passage of time, but it is also a dimension, in that it records when a given event happened. The key when examining any given quantity is to ask whether you would ever aggregate it (sum, average, etc.) or just cite it. In a basketball game it is the latter, so we separate it out into the play segment dimension. The clock *interval* between events (elapsed time), however, is something that we can sum up by quarter, player, etc. Thus, it belongs on the fact table.  
 


### **Do we really need dimension *tables*?**
One of the advantages of keeping dimensional data in separate tables is that it can significantly reduce storage costs by eliminating redundant data. However, with the advent of cloud-based data storage at commodity prices, storage becomes less important than performance. Thus, we may choose to denormalize everything into a single table that doesn't require any expensive joins. It's simple enough. If we already have the data in a normalized form, then we would just need to join in every table and select every column to generate a new "one table fits all" data warehouse. The dimensions would still be there, just as columns instead of tables.

While that sounds great in theory, in practice there are two good reasons for creating separate dimension tables: 
- **If care is taken to *use just foreign keys in the `GROUP BY` clause*, then it can actually be faster to query multiple tables than a single table.** This is because the joins will happen **after** the grouping has reduced the data to a manageable number of fact table rows. The incremental performance cost of the join is then practically nil, especially if there are a small number of groups in the result set. 
- **Dimension tables provide opportunities to add in static descriptive data.** For example, we could add in the seating capacity or age of a given basketball arena if we treat it as a dimension table instead of just a column. 

Whether either of these advantages are relevant depends on the situation. As a general rule, unless you have a good reason not to, it is best to create dimension tables instead of dimension columns. 

### **Snowflakes and Galaxies**

The rule that dimensions be denormalized is more of a convenience than a law. The purpose is to make writing queries as simple and bug-free as possible. However, if we are ultimately going to denormalize the data using the same joins every time, then what difference does it make how many joins there are? Such is the reasoning behind the so-called **Snowflake design pattern**. 

Here, for example, is the snowflake version of the PlayFacts database. The dotted lines show relationships normalized out from the star schema dimensions. There is no effect on the fact table, just more detail in the dimensions. 
![](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L10_NBA_PlayFacts_Snowflake_DW.png) 

The advantages of normalizing the dimensions include:
- less data redundancy, smaller storage requirements, etc. that we usually associate with normalization
- the ability to add new details (like team franchises or arenas) without disrupting existing dimensions

It is important to point out that the snowflake and star schema patterns are both perfectly valid. Which to use is totally situational. Let the data, its usage, and performance considerations guide your decision. 

Like the snowflake pattern, the **galaxy pattern** also extends the star schema, this time by allowing multiple fact tables. The baseball database from lesson 2 is a galaxy with ...
- Fact tables for fielding, batting, pitching, etc.  
- Dimension tables for players (Master), teams, all star game appearances, etc. 
It also follows the snowflake pattern, with enhancements like All Star Game appearances, Hall of Fame voting, etc. attached to the players and teams. 

![Lahman 2016 ERD](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L2_baseball_stats_schema.png)


---
## **Rollups and Drilldowns**

One of the big advantages of a dimensional data warehouse design is that it make it very simple to aggregate and disaggregate data at various levels of granularity. 

> **Heads Up**: The terms "rollup" and "drilldown" were coined by vendors of OLAP (OnLine Analytical Processing) systems that do real-time ETL from transaction data but the terms can be readily applied to any dimensional data warehouse.

A **rollup** is the standard `GROUP BY` aggregation operation. The idea is that a whole stripe of data in the cube (i.e., a dimension) is "rolled up" like a carpet and then replaced with summary data. We can do this for several dimensions at a time to get summarized data of various purposes. 

A **drilldown** is the opposite of a rollup. Starting with aggregate data, a drilldown disaggregates it to a finer level of granularity. Behind the scenes, it is the same as a rollup, just with a lower level of aggregation. Ultimately, the lowest level of a drilldown is the fact table itself. 

---
## **NBA LineupFacts Data Mart**

A data mart is most typically generated from two data sources:
- A custom rollup extracted from the data warehouse
- Supplementary data and calculations provided by the analysts 

The `lineup_facts` dataset from Lesson 3 (see below) was created to support a research project on the effect of teamwork in basketball. It includes a fully denormalized table (facts w / dimension columns) `play_facts_all`, from which we can calculate various statistics about the usage and efficiencies of basketball lineups over the years. 

![](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L3_GCP_data_explorer.png)

Before importing into Tableau in Lesson 9 we created a `segment_facts` table that rolls up the *play segments* to *lineup segments*. A lineup segment is a sequence of play segments where neither lineup (home or away) changes. For a game with 500+ play segments, there might be 25 or fewer lineup segments, with each segment lasting an average of about 2 minutes. 

However, even though the `segment_facts` table is 20x smaller than the original `play_facts_all` table, it is still too large to fit into MS Excel or similar desktop software. For that there is the `season_facts_all` table, which rolls the `segment_facts` table by lineup and year. It *just barely* fits into MS Excel, with just over 200K rows and 35 columns. Besides rolling up the original `play_facts_all` columns, the `season_facts_all` table includes a new column, `plus_minus_36m`, that is needed for the research questions that motivated the study. 

![Season Facts in Excel](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L10_Lineup_Season_Facts.png)














---
## **PRO TIPS: How to work with longitudinal data**

Longitudinal data, where the meaning of a given fact is determined relative to the ones before it, is inherently problematic. The complexity is subtle, practically invisible to untrained eyes, but it can nonetheless wreak havoc on even the best data model. 

The relational data model is designed to capture instantaneous snapshots of whatever is being tracked, adding one row at a time. 
```sql
INSERT INTO table (columns)
  rows;
```

Similarly, `SELECT` queries return data one row at a time. That makes it nice and symmetric, with rows in and rows out. 

Now for the subtle part. If we want to include a calculated column, say the difference between two other columns in the same row, then we'd do something like this:
```sql
SELECT column_b - column_a AS b_minus_a 
FROM ... ;
```
That seems simple enough. We can also, of course, use `GROUP BY` to do totals of a given column if we like, but the basic idea is still the same.  

However, what if we need to calculate differences between each row and *the one above it*? Then it gets more complicated. Not even `GROUP BY` can help us with that, unless we somehow make each row and the one above it a group, which seems highly inefficient at best. We'll instead have to dig deeper into our bag of tricks: 
- Join a table to itself using some sort of lag logic on the keys:
```
SELECT t2.column_a - t1.column_a AS diff_a_lag_1
FROM table AS t1 JOIN table AS t2 ON (t1.id = t2.id + 1)
```
- Use Python or another programming language to do the work. pandas has the `shift()` method just for such a purpose. You merge one table with a shifted version of itself. If you go this route, of course, you will need SQL to read the input data and then write the calculations back to the database afterwards. That's a nontrivial solution to say the least.  
- Use a SQL **window function**, which operates vertically within a column instead of laterally within a row. 

For most use cases, window functions will be your best choice. 

Window functions are used in the `SELECT` and `ORDER BY` clauses to allow longitudinal aggregates to be included *without a `GROUP BY` clause*. Instead of the traditional grouping operation, the expression being calculated specifies a range of rows, anchored by the current row. The syntax is 
```sql
expression OVER ( window_selector )
```
The `window_selector` specifies what rows to include and can use pretty sophisticated selection criteria like any of the following:
- X rows before and Y rows after the current row
- All rows before the current row
- All rows where column X is within some range of Z in the current row
- X rows before and Y rows after, sorted by the value of Z

The window function partitions the rows (much like `GROUP BY`) based on the given criteria and then calculates the expression based on the current row. The expression is almost always an aggregation function like `SUM()` or `AVG()`. of which there are too many to list here. There are even special ones like [`STRING_AGG()`](https://cloud.google.com/bigquery/docs/reference/standard-sql/functions-and-operators#string_agg) that work with non-numeric data types. 

Here, for example, we are calculating the cumulative minutes for each NBA player lineup throughout a season of play.  





In [ ]:
# load the bigquery magics extension
%load_ext google.cloud.bigquery
%load_ext google.colab.data_table

# authenticate for GCP access
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery
Authenticated


In [ ]:
%%bigquery --project nba-lineup-facts
SELECT 
    game_id, 
    date,
    lineup, 
    sum(play_length_mins) OVER (PARTITION BY lineup ORDER BY game_id ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW ) AS cum_mins
FROM `nba-lineup-facts.lineup_facts.game_facts_all` 
WHERE year = 2020
ORDER BY lineup, date
LIMIT 100

,game_id,date,lineup,cum_mins
0,21900415,2019-12-18,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",0.083333
1,21900665,2020-01-22,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",0.700000
2,21900679,2020-01-24,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",4.233333
3,21900757,2020-02-05,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",13.450000
4,21900757,2020-02-05,"[""D'Angelo Russell"", 'Damion Lee', 'Draymond G...",1.733333
...,...,...,...,...
95,21900602,2020-01-14,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",2.666667
96,21900588,2020-01-12,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",1.500000
97,21900602,2020-01-14,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",2.183333
98,21900076,2019-11-01,"[""D'Angelo Russell"", 'Draymond Green', 'Glenn ...",3.216667


The window specification is the part inside the second parentheses:
```sql 
PARTITION BY lineup 
ORDER BY game_id 
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW 
```
- `PARTITION BY` works like a `GROUP BY` to split the rows into groups/partitions.
- `ORDER BY` does as expected, but always with respect to the partition.
- `ROWS` specifies the width of the window; in this case that means everything up to the current row in the current partition (lineup).

There are lots of varieties to play with. For details read the docs for your database vendor. The SQL standard defines the basics like `PARTITION BY`, `ORDER BY`, and `ROWS` but there are some differences for more advanced use cases.  

> **Heads Up**: Although window functions were included in SQL:2003, most of the vendors only recently implemented the standard.  Check to see if your DBMS supports them before trying to use them.  

---
## **SQL AND BEYOND: Collection Types**

The **Collection** data types `ARRAY` and `STRUCT` extend SQL to allow multiple items to be stored in a single column. 

Remember this from Lesson 5, where a repeating field was used to store the movie credits?  

![Lesson 5 format 1](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L5_Movies_Tonight_format1.png)

Collections allow us to store these sorts of repeating fields directly, without normalizing into separate tables. 

Collection types were added to the SQL standards in a wave of enthusiasm for object-relational features in SQL:1999 but then (like window functions) largely ignored by most database vendors until 2018. Why ignore the standard? Because collection types permit us to skirt around a fundamental law of normalization: thou shalt not use repeated fields. 

In fact, it is not a good idea to use these sorts of collection types in a transactional database, where vigilence is needed to keep the data consistent and anomaly free. However, for dimensional data warehouses, where the **dimension tables** are often denormalized anyway, they make **perfect sense**: no extra joins, no redundant columns, etc., just the relevant context in a concise format.  

> **Heads Up**: Collection types are generally **not a good fit for fact tables** because they complicate the aggregation logic. What does a **measure** (i.e., fact) mean when it can be multi-valued? How do you sum or average a list of lists? 

***Technically*, collection types do not actually violate 1NF anyway.** A collection is a **container** that can hold structured data. The container itself is the "one thing" being stored in the column. It's like how you might keep a *container* of cut celery sticks in the fridge instead of storing them in a pile. The container is one thing (with the sticks nested inside), but you have open the container (unpack it) to get your snack. That's exactly how SQL `ARRAY` and `STRUCT` work. We have to use functions and special syntax to pack and unpack data. Otherwise, SQL just treats it like an image or other binary data. 

Now let's go back to the NBA data to see how it works. To keep things simple we will just focus on the lineup dimension, which in our play log data is kept in columns `a1`-`a5` and `h1`-`h5`.

   


### **The `ARRAY` Data Type**

An `ARRAY` is an list of items where:
- The items are ordered, one after another, like the rows of a table.  
- All items have the same data type. If the first item is an integer, then so must every other item. 
- There can be any number of items. 

Note that these same rules also apply to table columns, where each column has a data type, is ordered by row, and there is no limit on how many items (rows) there are. This is by design, so that the same logic can apply.  

The query below returns two rows of play log data, with each lineup as an **`ARRAY`**.

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT
  [a1,a2,a3,a4,a5] as away_lineup, [h1,h2,h3,h4,h5] as home_lineup
FROM `nba-lineup-facts.lineup_facts.play_log_sample`
WHERE play_id IN (86,88);

,away_lineup,home_lineup
0,"[OG Anunoby, Marc Gasol, Kyle Lowry, Fred VanV...","[Gordon Hayward, Jaylen Brown, Kemba Walker, R..."
1,"[OG Anunoby, Marc Gasol, Kyle Lowry, Fred VanV...","[Gordon Hayward, Kemba Walker, Robert Williams..."


At first glance, this looks a lot like how we have already been representing lineups using text strings:  
```sql
['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet']
```

versus

```sql
OG Anunoby,Pascal Siakam,Marc Gasol,Kyle Lowry,Fred VanVleet
```

The first one, is actually a serialized (string) representation of a Python list (or JavaScript array), a string of characters with Python syntax included. The second one, the SQL `ARRAY`, is an a list of strings that can be addressed separately by position. It doesn't *represent* a list. It *is* a list, with all the flexibility that implies. 

To see why that might matter, compare these two examples.


In [ ]:
%%bigquery --project nba-lineup-facts

-- The lineup as string version
-- Does the lineup string include the text 'Marc Gasol'?
SELECT "['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet']" LIKE '%Marc Gasol%'

,f0_
0,True


In [ ]:
%%bigquery --project nba-lineup-facts

-- the lineup as ARRAY version
-- Does the player 'Marc Gasol' appear in the lineup array? 
SELECT 'Marc Gasol' IN UNNEST(['OG Anunoby','Pascal Siakam','Marc Gasol','Kyle Lowry','Fred VanVleet'])

,f0_
0,True


Both return the same result but work very differently. The first query works by looking for the text `'Marc Gasol'` within a lineup string, while the second query looks for *Marc Gasol's name* in the lineup array. That's much more natural, isn't it? Further, the array is much more flexible. We could, for example, replace every player's name with a number without generating and searching though lineup strings.  

There is another difference as well. When retrieving an `ARRAY` as a list we use the `UNNEST` function. As mentioned in the discussion of collections that opened this section, this lets us get around the relational model's rule that each column contain scalar (singular) values. The array is stored as a *container* for list items that needs to be *unpacked* before we can use it. The container is a singular thing that happens to hold more things.  

> **Heads Up**: We can also [create arrays from subqueries using the `ARRAY()` function](https://cloud.google.com/bigquery/docs/reference/standard-sql/array_functions), which opens up lots of possibilities. Basically, any column of data that we can select or construct can be inserted into an array. 
 

### **The `STRUCT` Data Type**

A `STRUCT` (short for Structured Array) is like a one-row mini-table, with named *fields* that can mix data of various types. Some call it the swiss army knife of data types. Anything we can store in a table row (including `ARRAY` data) can also be stored in a structured array.

In [ ]:
%%bigquery --project nba-lineup-facts
SELECT
  STRUCT(a1 AS p1, a2 AS p2, a3 AS  p3,a4 AS p4,a5 AS p5) AS away_lineup,
  STRUCT(h1 AS p1, h2 AS p2, h3 AS  p3,h4 AS p4,h5 AS p5) AS home_lineup
FROM `nba-lineup-facts.lineup_facts.play_log_sample`
WHERE play_id IN (86,88);

,away_lineup,home_lineup
0,"{'p1': 'OG Anunoby', 'p2': 'Marc Gasol', 'p3':...","{'p1': 'Gordon Hayward', 'h2': 'Jaylen Brown',..."
1,"{'p1': 'OG Anunoby', 'p2': 'Marc Gasol', 'p3':...","{'p1': 'Gordon Hayward', 'h2': 'Kemba Walker',..."


Each `STRUCT` works more or less like a `SELECT` clause:
- The fields are defined like column specs, with an aliases used to name the fields.
- Unlike the `ARRAY` type's shorthand `[ ]` notation, `STRUCT` collections are created by calling a function, `STRUCT()` that assembles the data into the structured array.
- Each field can store anything that can be expressed as an expression. The expression is only evaluated once, just before the data is written to the field.

Again, a structured aray is like a one-row table, with all the sophistication that implies. All it is missing is primary and foreign keys.

> **Heads Up**: The notation used by SQL to represent a structured array is basically identical to the Python `dict` and the JavaScript `object`. 

To unpack a structured array we use `column.field` dot notation. The example below packs players into home and away lineups (via a CTE `WITH` clause) and then unpacks them to recreate the `h1`-`h5` and `a1`-`a5` columns in the original play log dataset. 


In [36]:
%%bigquery --project nba-lineup-facts
WITH play_lineups AS  
  ( SELECT
      play_id,
      STRUCT(a1 AS p1, a2 AS p2, a3 AS  p3,a4 AS p4,a5 AS p5) AS away_lineup,
      STRUCT(h1 AS p1, h2 AS p2, h3 AS  p3,h4 AS p4,h5 AS p5) AS home_lineup
    FROM `nba-lineup-facts.lineup_facts.play_log_sample`
    WHERE play_id IN (86,88)
  )
SELECT home_lineup.p1 as h1, home_lineup.p2 as h2, home_lineup.p3 as h3, home_lineup.p4 as h4, home_lineup.p5 as h5,
       away_lineup.p1 as a1, away_lineup.p2 as a2, away_lineup.p3 as a3, away_lineup.p4 as a4, away_lineup.p5 as a5
FROM play_lineups


,h1,h2,h3,h4,h5,a1,a2,a3,a4,a5
0,Gordon Hayward,Jaylen Brown,Kemba Walker,Robert Williams III,Marcus Smart,OG Anunoby,Marc Gasol,Kyle Lowry,Fred VanVleet,Norman Powell
1,Gordon Hayward,Kemba Walker,Robert Williams III,Marcus Smart,Grant Williams,OG Anunoby,Marc Gasol,Kyle Lowry,Fred VanVleet,Norman Powell


### **Combining `ARRAY` and `STRUCT` to make complex data types**

We can combine `ARRAY` and `STRUCT` data to create arbitrarily complex data structures. The result is **trees** of data, with collections nested inside collections inside of collections ...

We saw an example in Lesson 6 when we explored JSON formated data. The JSON fields shown below with arrowheads are collections, which have even more colelctions nested inside:
![](https://github.com/christopherhuntley/BUAN6510/raw/master/img/L6_Space_Jam_JSON.png)


To show how this is done in SQL, we will explore two techniques for creating *pseudo*-tables using nested collections:
- Create each column as an `ARRAY` and then nesting them inside a `STRUCT`
- Create each row as a `STRUCT` and then nesting them inside an `ARRAY`

In both cases the result is a single value that is functionally equivalent to an entire table. 

> **Heads Up**: The examples shown below are not intended to be of much practical use *unless* you need to store and parse JSON data in a relational database. 









#### **Nesting `ARRAY` columns inside a `STRUCT` pseudo-table**

In [49]:
%%bigquery --project nba-lineup-facts
SELECT STRUCT(
  ARRAY(SELECT play_id FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as play_id,
  ARRAY(SELECT a1 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a1,
  ARRAY(SELECT a2 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a2,
  ARRAY(SELECT a3 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a3,
  ARRAY(SELECT a4 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a4,
  ARRAY(SELECT a5 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as a5,
  ARRAY(SELECT a1 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h1,
  ARRAY(SELECT a2 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h2,
  ARRAY(SELECT a3 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h3,
  ARRAY(SELECT a4 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h4,
  ARRAY(SELECT a5 FROM `nba-lineup-facts.lineup_facts.play_log_sample` WHERE play_id in (86,88)) as h5 
) AS pseudo_table_by_cols

,pseudo_table_by_cols
0,"{'play_id': [86, 88], 'a1': ['OG Anunoby', 'OG..."


#### **Nesting `STRUCT` rows inside an `ARRAY` pseudo-table**

In [47]:
%%bigquery --project nba-lineup-facts
SELECT ARRAY(
  SELECT STRUCT(play_id, a1,a2,a3,a4,a5, h1,h2,h3,h4,h5)
  FROM `nba-lineup-facts.lineup_facts.play_log_sample`
  WHERE play_id in (86,88) 
) AS pseudo_table_by_rows

,pseudo_table_by_rows
0,"[{'play_id': 86, 'a1': 'OG Anunoby', 'a2': 'Ma..."


---
## **Congratulations! You've made it to the end of Lesson 10.**

Next week we will consider alternative data models that can improve flexibility and performance. 



## **On your way out ... Be sure to save your work**.
In Google Drive, drag this notebook file into your `BUAN6510` folder so you can find it next time.